<a href="https://colab.research.google.com/github/bmnds/uea-data-science-04-data-processing/blob/main/trabalho-final/Notebook_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
!pwd

/content/gdrive/MyDrive/uea-data-science/04-data-processing/trabalho-final/datasets


In [91]:
# Importando libs necessárias
import pandas as pd

# Extração

In [92]:
# Montar o diretório compartilhado do Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive


In [93]:
# Listar arquivos do diretório compartilhado
%cd MyDrive/uea-data-science/04-data-processing/trabalho-final/datasets
!ls

/content/gdrive/MyDrive/uea-data-science/04-data-processing/trabalho-final/datasets
bweb_1t_AM_181120201549.zip
bweb_2t_AM_301120201245.zip
detalhe_votacao_secao_2020.zip
eleitorado_local_votacao_2020.zip
perfil_comparecimento_abstencao_2020.zip
perfil_eleitorado_2020.zip
perfil_eleitor_secao_2020_AM.zip
prestacao_contas_anual_partidaria_2020.zip
prestacao_de_contas_eleitorais_candidatos_2020.zip
votacao_candidato_munzona_2020.zip
votacao_secao_2020_AM.zip


# Transformação

In [94]:
!mkdir /content/work #diretório temporário de trabalho
!mkdir /content/datasets #diretório com os arquivos processados

mkdir: cannot create directory ‘/content/work’: File exists
mkdir: cannot create directory ‘/content/datasets’: File exists


## Dados de Boletins de Urna

In [95]:
# boletins de urna do 1o turno
!unzip -u bweb_1t_AM_181120201549.zip -d /content/work

Archive:  bweb_1t_AM_181120201549.zip
  inflating: /content/work/bweb_1t_AM_181120201549.csv  
  inflating: /content/work/leiame-boletimurnaweb.pdf  


In [96]:
!cp /content/work/leiame-boletimurnaweb.pdf /content/datasets/leiame-boletim_urna.pdf

In [97]:
df_bu_1t = pd.read_csv('/content/work/bweb_1t_AM_181120201549.csv', encoding="ISO-8859-1", delimiter=';')
df_bu_1t.sample(5)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_PLEITO,DT_PLEITO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,CD_CARGO_PERGUNTA,DS_CARGO_PERGUNTA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,DT_BU_RECEBIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,CD_TIPO_URNA,DS_TIPO_URNA,CD_TIPO_VOTAVEL,DS_TIPO_VOTAVEL,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_URNA_EFETIVADA,CD_CARGA_1_URNA_EFETIVADA,CD_CARGA_2_URNA_EFETIVADA,CD_FLASHCARD_URNA_EFETIVADA,DT_CARGA_URNA_EFETIVADA,DS_CARGO_PERGUNTA_SECAO,DS_AGREGADAS,DT_ABERTURA,DT_ENCERRAMENTO,QT_ELEITORES_BIOMETRIA_NH,DT_EMISSAO_BU,NR_JUNTA_APURADORA,NR_TURMA_APURADORA
610924,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,AM,2550,MANAUS,62,369,1740,13,Vereador,10,REPUBLICANOS,REPUBLICANOS,15/11/2020 18:56:17,397,350,47,1,APURADA,1,Nominal,10300,FRED MOTA,1,1783585,879.551.552.820.767.341.,462.993,9E8EFA63,07/11/2020 08:55:00,13 - 369,#NULO#,15/11/2020 07:00:00,15/11/2020 17:01:53,0,15/11/2020 17:02:01,-1,-1
842238,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,AM,2550,MANAUS,70,763,1830,13,Vereador,33,PMN,Partido da Mobilização Nacional,15/11/2020 20:48:41,387,296,91,1,APURADA,1,Nominal,33777,ROSINALDO BUAL,2,1179705,637.112.013.151.473.700.,823.337,AB6F52C9,10/11/2020 16:06:00,13 - 763,#NULO#,15/11/2020 07:00:48,15/11/2020 17:03:44,0,15/11/2020 17:05:14,-1,-1
288375,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,AM,2550,MANAUS,31,698,2089,13,Vereador,20,PSC,Partido Social Cristão,15/11/2020 18:40:06,565,456,109,1,APURADA,1,Nominal,20999,CHIQUINHO AMARAL,3,1181181,374.569.425.465.741.193.,814.757,A9FEAB10,05/11/2020 09:05:00,13 - 698,747,15/11/2020 07:00:00,15/11/2020 17:04:45,0,15/11/2020 17:07:16,-1,-1
152019,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,AM,2534,MANACAPURU,6,204,1511,13,Vereador,11,PP,PROGRESSISTAS,15/11/2020 20:55:17,393,339,54,1,APURADA,1,Nominal,11236,DANILO PARAÍBA,1,1789148,951.761.574.541.146.998.,92.771,46FC6683,06/11/2020 13:27:00,13 - 204,#NULO#,15/11/2020 07:00:00,15/11/2020 17:04:28,0,15/11/2020 17:06:22,-1,-1
215454,18/11/2020,15:49:53,2020,0,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020,AM,2356,HUMAITÁ,17,4,1023,13,Vereador,10,REPUBLICANOS,REPUBLICANOS,15/11/2020 20:46:22,365,280,85,1,APURADA,1,Nominal,10456,SOCORRO FIGUEIREDO,1,1264650,540.238.987.617.469.454.,309.862,99616BE5,06/11/2020 10:54:00,13 - 4,#NULO#,15/11/2020 07:00:00,15/11/2020 17:02:38,0,15/11/2020 17:04:15,-1,-1


In [98]:
df_bu_1t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 848941 entries, 0 to 848940
Data columns (total 45 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   DT_GERACAO                   848941 non-null  object 
 1   HH_GERACAO                   848941 non-null  object 
 2   ANO_ELEICAO                  848941 non-null  int64  
 3   CD_TIPO_ELEICAO              848941 non-null  int64  
 4   NM_TIPO_ELEICAO              848941 non-null  object 
 5   CD_PLEITO                    848941 non-null  int64  
 6   DT_PLEITO                    848941 non-null  object 
 7   NR_TURNO                     848941 non-null  int64  
 8   CD_ELEICAO                   848941 non-null  int64  
 9   DS_ELEICAO                   848941 non-null  object 
 10  SG_UF                        848941 non-null  object 
 11  CD_MUNICIPIO                 848941 non-null  int64  
 12  NM_MUNICIPIO                 848941 non-null  object 
 13 

In [99]:
df_bu_1t.DS_CARGO_PERGUNTA.unique()

array(['Prefeito', 'Vereador'], dtype=object)

In [100]:
df_bu_1t.SG_PARTIDO.value_counts()

PSC              62642
AVANTE           60226
REPUBLICANOS     53694
PL               40921
PATRIOTA         40139
PP               36335
PT               34365
PSDB             33885
PODE             33417
PSL              33355
PTB              32379
MDB              31878
PMN              30224
PSB              29228
PSD              27816
DC               26548
CIDADANIA        26338
PROS             26234
PV               25449
#NULO#           24702
DEM              24324
PRTB             21962
PTC              21249
SOLIDARIEDADE    17075
PC do B          16626
PDT              12287
PMB               9386
NOVO              7568
PSOL              4880
REDE              2877
PSTU               932
Name: SG_PARTIDO, dtype: int64

In [101]:
df_bu_1t.NM_MUNICIPIO.value_counts()

MANAUS         590874
MANACAPURU      20977
ITACOATIARA     20787
PARINTINS       18377
TEFÉ            12425
                ...  
AMATURÁ          1116
ANAMÃ            1092
CANUTAMA         1033
ITAMARATI         800
JAPURÁ            583
Name: NM_MUNICIPIO, Length: 62, dtype: int64

In [102]:
df_bu_1t.DS_TIPO_VOTAVEL.unique()

array(['Nominal', 'Nulo', 'Branco', 'Legenda'], dtype=object)

In [103]:
df_bu_1t.DS_TIPO_URNA.unique()

array(['APURADA'], dtype=object)

In [104]:
df_bu_1t.DS_AGREGADAS.value_counts()

#NULO#     805387
80            585
73            551
50            467
61            423
            ...  
90             24
17 / 26        19
65             19
32             17
111            14
Name: DS_AGREGADAS, Length: 294, dtype: int64

In [105]:
df_bu_1t = df_bu_1t[df_bu_1t.NM_MUNICIPIO.eq('MANAUS')] \
                  .drop(columns=['DT_GERACAO',	'HH_GERACAO',	'CD_TIPO_ELEICAO', 'DT_BU_RECEBIDO',	'CD_TIPO_URNA', 'DS_TIPO_URNA',	
                                    'CD_TIPO_VOTAVEL',	'NR_URNA_EFETIVADA',	'CD_CARGA_1_URNA_EFETIVADA',	'CD_CARGA_2_URNA_EFETIVADA',	
                                    'CD_FLASHCARD_URNA_EFETIVADA',	'DT_CARGA_URNA_EFETIVADA',	'DS_CARGO_PERGUNTA_SECAO',	'DS_AGREGADAS',
                                    'DT_ABERTURA',	'DT_ENCERRAMENTO',	'QT_ELEITORES_BIOMETRIA_NH',	'NR_JUNTA_APURADORA',	'NR_TURMA_APURADORA'])
df_bu_1t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 590874 entries, 0 to 848940
Data columns (total 26 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ANO_ELEICAO        590874 non-null  int64 
 1   NM_TIPO_ELEICAO    590874 non-null  object
 2   CD_PLEITO          590874 non-null  int64 
 3   DT_PLEITO          590874 non-null  object
 4   NR_TURNO           590874 non-null  int64 
 5   CD_ELEICAO         590874 non-null  int64 
 6   DS_ELEICAO         590874 non-null  object
 7   SG_UF              590874 non-null  object
 8   CD_MUNICIPIO       590874 non-null  int64 
 9   NM_MUNICIPIO       590874 non-null  object
 10  NR_ZONA            590874 non-null  int64 
 11  NR_SECAO           590874 non-null  int64 
 12  NR_LOCAL_VOTACAO   590874 non-null  int64 
 13  CD_CARGO_PERGUNTA  590874 non-null  int64 
 14  DS_CARGO_PERGUNTA  590874 non-null  object
 15  NR_PARTIDO         590874 non-null  int64 
 16  SG_PARTIDO         5

### Candidatos a Prefeito

In [106]:
df_prefeitos = df_bu_1t[df_bu_1t.DS_CARGO_PERGUNTA.eq('Prefeito') & df_bu_1t.DS_TIPO_VOTAVEL.eq('Nominal')] \
                      [['CD_ELEICAO','DS_CARGO_PERGUNTA','NR_VOTAVEL','SG_PARTIDO','NM_VOTAVEL']] \
                         .drop_duplicates() \
                         .sort_values(['NR_VOTAVEL'])
df_prefeitos.columns = ['eleicao',	'cargo',	'numero', 'partido', 'candidato']
df_prefeitos

,eleicao,cargo,numero,partido,candidato
6,426,Prefeito,10,REPUBLICANOS,CAPITÃO ALBERTO NETO
9,426,Prefeito,13,PT,ZE RICARDO
209,426,Prefeito,16,PSTU,GILBERTO VASCONCELOS
7,426,Prefeito,19,PODE,AMAZONINO MENDES
5,426,Prefeito,22,PL,ALFREDO NASCIMENTO
0,426,Prefeito,27,DC,CHICO PRETO
10,426,Prefeito,30,NOVO,ROMERO REIS
8,426,Prefeito,51,PATRIOTA,CORONEL MENEZES
4,426,Prefeito,55,PSD,RICARDO NICOLAU
217,426,Prefeito,65,PC do B,MARCELO AMIL


In [107]:
df_prefeitos.to_csv('/content/datasets/prefeitos.csv')

### Resultados para Prefeito

In [108]:
df_resultados_prefeito = df_bu_1t[df_bu_1t.DS_CARGO_PERGUNTA.eq('Prefeito')] \
    [['CD_ELEICAO',	'NR_ZONA'	,'NR_SECAO', 'NR_LOCAL_VOTACAO', 'DS_CARGO_PERGUNTA'	,'NR_VOTAVEL',	'QT_VOTOS']] \
    .sort_values(['NR_ZONA','NR_SECAO','NR_VOTAVEL'])
df_resultados_prefeito.columns = ['eleicao',	'zona',	'secao', 'local',	'cargo',	'numero',	'votos']    
df_resultados_prefeito

,eleicao,zona,secao,local,cargo,numero,votos
6,426,1,640,2429,Prefeito,10,19
9,426,1,640,2429,Prefeito,13,49
7,426,1,640,2429,Prefeito,19,90
5,426,1,640,2429,Prefeito,22,7
0,426,1,640,2429,Prefeito,27,7
...,...,...,...,...,...,...,...
848806,426,70,808,1848,Prefeito,51,13
848799,426,70,808,1848,Prefeito,55,19
848804,426,70,808,1848,Prefeito,70,46
848796,426,70,808,1848,Prefeito,95,9


In [109]:
filename = '/content/datasets/resultados_prefeito.csv'
df_resultados_prefeito.to_csv(filename)
print(f'Resultado da votação para prefeito no 1o turno gravada em \'{filename}\'')

Resultado da votação para prefeito no 1o turno gravada em '/content/datasets/resultados_prefeito.csv'


### Candidatos a Vereador

In [110]:
df_vereadores = df_bu_1t[df_bu_1t.DS_CARGO_PERGUNTA.eq('Vereador') & df_bu_1t.DS_TIPO_VOTAVEL.eq('Nominal')] \
                        [['CD_ELEICAO','DS_CARGO_PERGUNTA','NR_VOTAVEL','SG_PARTIDO','NM_VOTAVEL']] \
                         .drop_duplicates() \
                         .sort_values(['NR_VOTAVEL'])
df_vereadores.columns = ['eleicao',	'cargo',	'numero', 'partido', 'candidato']
df_vereadores


,eleicao,cargo,numero,partido,candidato
3083,426,Vereador,10000,REPUBLICANOS,CABO COUTINHO
32033,426,Vereador,10007,REPUBLICANOS,SARGENTO ANA CLAUDIA
36326,426,Vereador,10008,REPUBLICANOS,CHAY CHANTEL
4547,426,Vereador,10010,REPUBLICANOS,LUCAS SALES
453,426,Vereador,10018,REPUBLICANOS,JOSUE DOS CAMINHONEIROS
...,...,...,...,...,...
950,426,Vereador,90800,PROS,EMERSON BOTELHO
10173,426,Vereador,90888,PROS,LORA DO SOCIAL
1335,426,Vereador,90900,PROS,ERICO BRAGA
1615,426,Vereador,90901,PROS,LUIZ ALBERTO


In [111]:
df_vereadores.to_csv('/content/datasets/vereadores.csv')

### Resultados para Vereador

In [112]:
df_resultados_vereador = df_bu_1t[df_bu_1t.DS_CARGO_PERGUNTA.eq('Vereador')] \
    [['CD_ELEICAO',	'NR_ZONA'	,'NR_SECAO', 'NR_LOCAL_VOTACAO', 'DS_CARGO_PERGUNTA'	,'NR_VOTAVEL',	'QT_VOTOS']] \
    .sort_values(['NR_ZONA','NR_SECAO','NR_VOTAVEL'])
df_resultados_vereador.columns = ['eleicao',	'zona',	'secao', 'local',	'cargo',	'numero',	'votos']    
df_resultados_vereador

,eleicao,zona,secao,local,cargo,numero,votos
200,426,1,640,2429,Vereador,11,1
104,426,1,640,2429,Vereador,13,2
185,426,1,640,2429,Vereador,18,1
25,426,1,640,2429,Vereador,19,2
54,426,1,640,2429,Vereador,50,1
...,...,...,...,...,...,...,...
848823,426,70,808,1848,Vereador,77888,4
848898,426,70,808,1848,Vereador,90177,2
848832,426,70,808,1848,Vereador,90321,1
848830,426,70,808,1848,Vereador,90523,1


In [113]:
filename = '/content/datasets/resultados_vereador.csv'
df_resultados_vereador.to_csv(filename)
print(f'Resultado da votação para Vereador no 1o turno gravada em \'{filename}\'')

Resultado da votação para Vereador no 1o turno gravada em '/content/datasets/resultados_vereador.csv'


### Candidatos a Prefeito (2o turno)

In [114]:
# boletins de urna do 2o turno
!unzip -u bweb_2t_AM_301120201245.zip -d /content/work

Archive:  bweb_2t_AM_301120201245.zip
  inflating: /content/work/bweb_2t_AM_301120201245.csv  


In [115]:
df_bu_2t = pd.read_csv('/content/work/bweb_2t_AM_301120201245.csv', encoding="ISO-8859-1", delimiter=';')
df_bu_2t.sample(5)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,CD_PLEITO,DT_PLEITO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,CD_CARGO_PERGUNTA,DS_CARGO_PERGUNTA,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,DT_BU_RECEBIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,CD_TIPO_URNA,DS_TIPO_URNA,CD_TIPO_VOTAVEL,DS_TIPO_VOTAVEL,NR_VOTAVEL,NM_VOTAVEL,QT_VOTOS,NR_URNA_EFETIVADA,CD_CARGA_1_URNA_EFETIVADA,CD_CARGA_2_URNA_EFETIVADA,CD_FLASHCARD_URNA_EFETIVADA,DT_CARGA_URNA_EFETIVADA,DS_CARGO_PERGUNTA_SECAO,DS_AGREGADAS,DT_ABERTURA,DT_ENCERRAMENTO,QT_ELEITORES_BIOMETRIA_NH,DT_EMISSAO_BU,NR_JUNTA_APURADORA,NR_TURMA_APURADORA
280,30/11/2020,12:37:06,2020,0,Eleição Ordinária,305,29/11/2020,2,427,Eleições Municipais 2020,AM,2550,MANAUS,1,716,2089,11,Prefeito,19,PODE,Podemos,29/11/2020 18:21:31,392,322,70,1,APURADA,1,Nominal,19,AMAZONINO MENDES,155,1787989,344.245.707.533.489.803.,358.558,22E484E7,10/11/2020 08:43:00,11 - 716,#NULO#,29/11/2020 07:00:00,29/11/2020 17:01:32,0,29/11/2020 17:03:22,-1,-1
12430,30/11/2020,12:37:06,2020,0,Eleição Ordinária,305,29/11/2020,2,427,Eleições Municipais 2020,AM,2550,MANAUS,70,445,1970,11,Prefeito,19,PODE,Podemos,29/11/2020 18:18:45,374,268,106,1,APURADA,1,Nominal,19,AMAZONINO MENDES,121,1774803,182.667.377.241.149.483.,809.126,A6A50266,10/11/2020 14:37:00,11 - 445,#NULO#,29/11/2020 07:00:00,29/11/2020 17:02:31,0,29/11/2020 17:05:01,-1,-1
3177,30/11/2020,12:37:06,2020,0,Eleição Ordinária,305,29/11/2020,2,427,Eleições Municipais 2020,AM,2550,MANAUS,32,754,2089,11,Prefeito,-1,#NULO#,#NULO#,29/11/2020 18:24:17,391,311,80,1,APURADA,3,Nulo,96,Nulo,22,1789740,829.025.240.284.322.504.,998.230,08144FBD,09/11/2020 15:07:00,11 - 754,#NULO#,29/11/2020 07:00:00,29/11/2020 17:04:52,0,29/11/2020 17:06:06,-1,-1
1586,30/11/2020,12:37:06,2020,0,Eleição Ordinária,305,29/11/2020,2,427,Eleições Municipais 2020,AM,2550,MANAUS,2,705,1953,11,Prefeito,70,AVANTE,AVANTE,29/11/2020 18:10:36,366,263,103,1,APURADA,1,Nominal,70,DAVID ALMEIDA,124,1179945,465.468.647.204.336.736.,820.992,12AEB082,06/11/2020 09:37:00,11 - 705,#NULO#,29/11/2020 07:00:18,29/11/2020 17:02:40,0,29/11/2020 17:03:59,-1,-1
13225,30/11/2020,12:37:06,2020,0,Eleição Ordinária,305,29/11/2020,2,427,Eleições Municipais 2020,AM,2550,MANAUS,70,740,1880,11,Prefeito,-1,#NULO#,#NULO#,29/11/2020 18:17:15,378,292,86,1,APURADA,3,Nulo,96,Nulo,9,1789545,960.445.874.771.647.018.,162.165,6103342A,10/11/2020 15:14:00,11 - 740,#NULO#,29/11/2020 07:00:00,29/11/2020 17:02:07,0,29/11/2020 17:02:56,-1,-1


In [116]:
df_prefeitos_2t = df_bu_2t[df_bu_2t.DS_CARGO_PERGUNTA.eq('Prefeito') & df_bu_2t.DS_TIPO_VOTAVEL.eq('Nominal')] \
                       [['CD_ELEICAO','DS_CARGO_PERGUNTA','SG_PARTIDO','NR_VOTAVEL','NM_VOTAVEL']] \
                         .drop_duplicates() \
                         .sort_values(['NR_VOTAVEL'])
df_prefeitos_2t.columns = ['eleicao',	'cargo',	'numero', 'partido', 'candidato']
df_prefeitos_2t

,eleicao,cargo,numero,partido,candidato
1,427,Prefeito,PODE,19,AMAZONINO MENDES
0,427,Prefeito,AVANTE,70,DAVID ALMEIDA


In [117]:
filename = '/content/datasets/prefeitos_2t.csv'
df_prefeitos_2t.to_csv(filename)
print(f'Candidatos a Prefeito (2o turno) gravados em \'{filename}\'')

Candidatos a Prefeito (2o turno) gravados em '/content/datasets/prefeitos_2t.csv'


### Resultados para Prefeito (2o turno)

In [118]:
df_resultado_prefeito = df_bu_2t[df_bu_2t.DS_CARGO_PERGUNTA.eq('Prefeito')] \
    [['CD_ELEICAO',	'NR_ZONA'	,'NR_SECAO', 'NR_LOCAL_VOTACAO', 'DS_CARGO_PERGUNTA'	,'NR_VOTAVEL',	'QT_VOTOS']] \
    .sort_values(['NR_ZONA','NR_SECAO','NR_VOTAVEL'])
df_resultado_prefeito.columns = ['eleicao',	'zona',	'secao', 'local',	'cargo',	'numero',	'votos']    
df_resultado_prefeito

,eleicao,zona,secao,local,cargo,numero,votos
1,427,1,640,2429,Prefeito,19,156
0,427,1,640,2429,Prefeito,70,135
3,427,1,640,2429,Prefeito,95,10
2,427,1,640,2429,Prefeito,96,29
4,427,1,641,2089,Prefeito,19,153
...,...,...,...,...,...,...,...
13471,427,70,806,2089,Prefeito,96,16
13476,427,70,808,1848,Prefeito,19,68
13473,427,70,808,1848,Prefeito,70,122
13475,427,70,808,1848,Prefeito,95,11


In [119]:
filename = '/content/datasets/resultados_prefeito_2t.csv'
df_resultado_prefeito.to_csv(filename)
print(f'Resultado da votação para Prefeito (2o turno) gravado em \'{filename}\'')

Resultado da votação para Prefeito (2o turno) gravado em '/content/datasets/resultados_prefeito_2t.csv'


### Eleições

In [120]:
df1 = df_bu_1t[['ANO_ELEICAO',	'NM_TIPO_ELEICAO',	'CD_PLEITO',	'DT_PLEITO', 'NR_TURNO',	'CD_ELEICAO',	'DS_ELEICAO']] \
               .drop_duplicates()
df2 = df_bu_2t[['ANO_ELEICAO',	'NM_TIPO_ELEICAO',	'CD_PLEITO',	'DT_PLEITO', 'NR_TURNO',	'CD_ELEICAO',	'DS_ELEICAO']] \
               .drop_duplicates()
df_eleicao = df1.append(df2)
df_eleicao.columns = ['ano', 'tipo', 'pleito', 'data', 'turno', 'eleicao', 'descricao']
df_eleicao

,ano,tipo,pleito,data,turno,eleicao,descricao
0,2020,Eleição Ordinária,304,15/11/2020,1,426,Eleições Municipais 2020
0,2020,Eleição Ordinária,305,29/11/2020,2,427,Eleições Municipais 2020


In [121]:
filename = '/content/datasets/eleicoes.csv'
df_resultado_prefeito.to_csv(filename)
print(f'Eleições gravadas em \'{filename}\'')

Eleições gravadas em '/content/datasets/eleicoes.csv'


### Limpeza do diretório de trabalho

In [122]:
# Limpeza do diretório temporário de trabalho
!rm -rf /content/work/*

## Locais de votação

In [123]:
# detalhes das seções
!unzip -u detalhe_votacao_secao_2020.zip -d /content/work

Archive:  detalhe_votacao_secao_2020.zip
  inflating: /content/work/detalhe_votacao_secao_2020_AC.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_AL.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_AM.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_AP.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_BA.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_BRASIL.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_CE.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_ES.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_GO.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_MA.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_MG.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_MS.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_MT.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_PA.csv  
  inflating: /content/work/detalhe_votacao_secao_2020_PB.csv  
  inflatin

In [124]:
df_dvs = pd.read_csv('/content/work/detalhe_votacao_secao_2020_BRASIL.csv', encoding="ISO-8859-1", delimiter=';')
df_dvs.sample(5)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_MUNICIPIO,NM_MUNICIPIO,NR_ZONA,NR_SECAO,CD_CARGO,DS_CARGO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,QT_VOTOS_NOMINAIS,QT_VOTOS_BRANCOS,QT_VOTOS_NULOS,QT_VOTOS_LEGENDA,QT_VOTOS_ANULADOS_APU_SEP,NR_LOCAL_VOTACAO
889620,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,2,427,ELEIÇÕES MUNICIPAIS 2020,29/11/2020,M,SP,71072,SÃO PAULO,71072,SÃO PAULO,260,360,11,PREFEITO,428,290,138,243,17,30,0,0,1295
804369,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,SP,70998,SÃO JOSÉ DOS CAMPOS,70998,SÃO JOSÉ DOS CAMPOS,412,264,13,VEREADOR,369,261,108,206,21,18,16,0,1520
732578,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,PR,78239,RIO NEGRO,78239,RIO NEGRO,11,95,13,VEREADOR,600,495,105,441,21,15,18,0,1066
760428,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,RJ,60011,RIO DE JANEIRO,60011,RIO DE JANEIRO,214,104,13,VEREADOR,391,241,150,177,23,22,19,0,1210
179937,09/12/2020,15:29:43,2020,2,ELEIÇÃO ORDINÁRIA,1,426,ELEIÇÕES MUNICIPAIS 2020,15/11/2020,M,BA,38830,SÃO FRANCISCO DO CONDE,38830,SÃO FRANCISCO DO CONDE,162,9,13,VEREADOR,369,336,33,320,2,7,7,0,1031


In [125]:
df_dvs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 897536 entries, 0 to 897535
Data columns (total 28 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   DT_GERACAO                 897536 non-null  object
 1   HH_GERACAO                 897536 non-null  object
 2   ANO_ELEICAO                897536 non-null  int64 
 3   CD_TIPO_ELEICAO            897536 non-null  int64 
 4   NM_TIPO_ELEICAO            897536 non-null  object
 5   NR_TURNO                   897536 non-null  int64 
 6   CD_ELEICAO                 897536 non-null  int64 
 7   DS_ELEICAO                 897536 non-null  object
 8   DT_ELEICAO                 897536 non-null  object
 9   TP_ABRANGENCIA             897536 non-null  object
 10  SG_UF                      897536 non-null  object
 11  SG_UE                      897536 non-null  int64 
 12  NM_UE                      897536 non-null  object
 13  CD_MUNICIPIO               897536 non-null  

### Seções

In [126]:
df_secoes = df_dvs[df_dvs.NM_MUNICIPIO.eq('MANAUS')] \
                  [['CD_ELEICAO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO',
                    'QT_APTOS',	'QT_COMPARECIMENTO',	'QT_ABSTENCOES']] \
                    .sort_values(['NR_ZONA', 'NR_SECAO', 'CD_ELEICAO']) \
                    .drop_duplicates()
df_secoes.columns = ['eleicao', 'zona', 'secao', 'local','aptos','votos','absencoes']
df_secoes

,eleicao,zona,secao,local,aptos,votos,absencoes
41373,426,1,640,2429,411,348,63
725917,427,1,640,2429,411,330,81
325496,426,1,641,2089,393,326,67
328669,427,1,641,2089,393,308,85
138660,426,1,642,2321,397,341,56
...,...,...,...,...,...,...,...
431441,427,70,805,2046,366,245,121
291560,426,70,806,2089,399,280,119
604720,427,70,806,2089,399,277,122
374725,426,70,808,1848,284,224,60


In [127]:
filename = '/content/datasets/secoes.csv'
df_secoes.to_csv(filename)
print(f'Seções eleitorais gravadas em \'{filename}\'')

Seções eleitorais gravadas em '/content/datasets/secoes.csv'


In [128]:
dft_dvs = df_dvs[df_dvs.NM_MUNICIPIO.eq('MANAUS')] \
               [['CD_ELEICAO', 'NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO', 'DS_CARGO',
                 'QT_APTOS', 'QT_COMPARECIMENTO', 'QT_ABSTENCOES',
                 'QT_VOTOS_NOMINAIS',	'QT_VOTOS_BRANCOS',	'QT_VOTOS_NULOS', 'QT_VOTOS_LEGENDA']] \
                .sort_values(['CD_ELEICAO','NR_ZONA', 'NR_SECAO'])
dft_dvs

,CD_ELEICAO,NR_ZONA,NR_SECAO,NR_LOCAL_VOTACAO,DS_CARGO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,QT_VOTOS_NOMINAIS,QT_VOTOS_BRANCOS,QT_VOTOS_NULOS,QT_VOTOS_LEGENDA
41373,426,1,640,2429,PREFEITO,411,348,63,314,15,19,0
205055,426,1,640,2429,VEREADOR,411,348,63,302,19,16,11
325496,426,1,641,2089,PREFEITO,393,326,67,293,10,23,0
340194,426,1,641,2089,VEREADOR,393,326,67,292,11,13,10
138660,426,1,642,2321,VEREADOR,397,341,56,306,12,17,6
...,...,...,...,...,...,...,...,...,...,...,...,...
891487,427,70,803,2003,PREFEITO,275,188,87,170,12,6,0
624649,427,70,804,2038,PREFEITO,332,210,122,183,12,15,0
431441,427,70,805,2046,PREFEITO,366,245,121,216,13,16,0
604720,427,70,806,2089,PREFEITO,399,277,122,242,19,16,0


### Escolas

In [129]:
# detalhes das seções
!unzip -u eleitorado_local_votacao_2020.zip -d /content/work

Archive:  eleitorado_local_votacao_2020.zip
  inflating: /content/work/LEIAME.pdf  
  inflating: /content/work/eleitorado_local_votacao_2020.csv  


In [130]:
df = pd.read_csv('/content/work/eleitorado_local_votacao_2020.csv', encoding="ISO-8859-1", delimiter=';')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 597429 entries, 0 to 597428
Data columns (total 32 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   DT_GERACAO              597429 non-null  object 
 1   HH_GERACAO              597429 non-null  object 
 2   AA_ELEICAO              597429 non-null  int64  
 3   DT_ELEICAO              597429 non-null  object 
 4   DS_ELEICAO              597429 non-null  object 
 5   SG_UF                   597429 non-null  object 
 6   CD_MUNICIPIO            597429 non-null  int64  
 7   NM_MUNICIPIO            597429 non-null  object 
 8   NR_ZONA                 597429 non-null  int64  
 9   NR_SECAO                597429 non-null  int64  
 10  CD_TIPO_SECAO_AGREGADA  597429 non-null  int64  
 11  DS_TIPO_SECAO_AGREGADA  597429 non-null  object 
 12  NR_LOCAL_VOTACAO        597429 non-null  int64  
 13  NM_LOCAL_VOTACAO        597429 non-null  object 
 14  CD_TIPO_LOCAL       

In [131]:
df_escolas = df[df.NM_MUNICIPIO.eq('MANAUS')] \
                  [['NR_ZONA', 'NR_SECAO', 'NR_LOCAL_VOTACAO', 'NM_LOCAL_VOTACAO',
                    'NM_BAIRRO','NR_LATITUDE',	'NR_LONGITUDE']] \
                    .sort_values(['NR_ZONA', 'NR_SECAO']) \
                    .drop_duplicates()
df_escolas.columns = ['zona', 'secao', 'local', 'nome', 'bairro', 'lat','long']
df_escolas

,zona,secao,local,nome,bairro,lat,long
3355,1,550,2291,E. M. PROF. WALDIR GARCIA,SAO GERALDO,-3.112827,-60.030421
366142,1,616,2399,E. E. RIBEIRO DA CUNHA,CENTRO,-3.123427,-60.023611
90957,1,640,2429,ESCOLA ESTADUAL CARVALHO LEAL (RECEBEU AS SEÇÕ...,CACHOEIRINHA,-1.000000,-1.000000
70444,1,641,2089,ESCOLA SUPERIOR DE CIENCIAS DA SAUDE - UEA,CACHOEIRINHA,-3.119252,-60.006131
179658,1,642,2321,E. E. MILBURGES BEZERRA DE ARAUJO,RAIZ,-3.123326,-59.995204
...,...,...,...,...,...,...,...
285905,70,816,2208,E. M. SOCIÓLOGO HERBERT JOSÉ SOUZA,NOVO ALEIXO,-1.000000,-1.000000
269250,70,817,2160,CMEI CALISTO PEREIRA CAVALCANTE,CIDADE DE DEUS,-1.000000,-1.000000
79432,70,819,2186,E. M. DR. ADERSON PEREIRA DUTRA,NOVO ALEIXO,-1.000000,-1.000000
74929,70,820,2216,CMEI SANTINA CASTRO PEREIRA,NOVO ALEIXO,-1.000000,-1.000000


In [132]:
filename = '/content/datasets/escolas.csv'
df_escolas.to_csv(filename)
print(f'Escolas gravadas em \'{filename}\'')

Escolas gravadas em '/content/datasets/escolas.csv'


In [133]:
# Limpeza do diretório temporário de trabalho
!rm -rf /content/work/*

## Perfil do Eleitorado
perfil_eleitor_secao_2020_AM.zip

In [ ]:
# perfil do eleitorado
!unzip -u perfil_eleitor_secao_2020_AM.zip -d /content/work

In [ ]:
df_perfil = pd.read_csv('/content/work/perfil_eleitor_secao_2020_AM.csv', encoding="ISO-8859-1", delimiter=';')
df_perfil.info()

In [9]:
df_perfil = df_perfil[df_perfil.NM_MUNICIPIO.eq('MANAUS')]
df_perfil.head(5)

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,CD_MUN_SIT_BIOMETRIA,DS_MUN_SIT_BIOMETRIA,NR_ZONA,NR_SECAO,CD_LOCAL_VOTACAO,CD_GENERO,DS_GENERO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
4039,01/08/2020,09:34:32,2020,AM,2550,MANAUS,0,Sem biometria,2,617,Ebc015012917371261,4,FEMININO,9,DIVORCIADO,5054,50 a 54 anos,8,SUPERIOR COMPLETO,1,1,0,0
4040,01/08/2020,09:34:32,2020,AM,2550,MANAUS,0,Sem biometria,2,617,Ebc015012917371261,4,FEMININO,9,DIVORCIADO,5559,55 a 59 anos,5,ENSINO MÉDIO INCOMPLETO,1,1,0,0
4041,01/08/2020,09:34:32,2020,AM,2550,MANAUS,0,Sem biometria,2,617,Ebc015012917371261,4,FEMININO,9,DIVORCIADO,5559,55 a 59 anos,8,SUPERIOR COMPLETO,1,1,0,0
4042,01/08/2020,09:34:32,2020,AM,2550,MANAUS,0,Sem biometria,2,617,Ebc015012917371261,4,FEMININO,9,DIVORCIADO,6569,65 a 69 anos,2,LÊ E ESCREVE,1,1,0,0
4043,01/08/2020,09:34:32,2020,AM,2550,MANAUS,0,Sem biometria,2,617,Ebc015012917371261,4,FEMININO,9,DIVORCIADO,6569,65 a 69 anos,8,SUPERIOR COMPLETO,1,1,0,0


# Carga

In [134]:
%cd /content
!tar -czvf eleicoes2020.tar.gz datasets/*

/content
datasets/eleicoes.csv
datasets/escolas.csv
datasets/leiame-boletim_urna.pdf
datasets/prefeitos_2t.csv
datasets/prefeitos.csv
datasets/resultados_prefeito_2t.csv
datasets/resultados_prefeito.csv
datasets/resultados_vereador.csv
datasets/secoes.csv
datasets/vereadores.csv


In [135]:
# sobrescreve o arquivo compartilhado: 1-1v_7xBem5VdYwFFaaNl2EKPLYaDadPa
!cp eleicoes2020.tar.gz /content/gdrive/MyDrive/uea-data-science/04-data-processing/trabalho-final

In [136]:
# remove o diretório
!rm -rf datasets

In [137]:
!tar -zxvf eleicoes2020.tar.gz

datasets/eleicoes.csv
datasets/escolas.csv
datasets/leiame-boletim_urna.pdf
datasets/prefeitos_2t.csv
datasets/prefeitos.csv
datasets/resultados_prefeito_2t.csv
datasets/resultados_prefeito.csv
datasets/resultados_vereador.csv
datasets/secoes.csv
datasets/vereadores.csv
